In [47]:
import pandas as pd
import glob
import os
from zipfile import ZipFile

In [48]:
# find the name of the zip file in the directory 
print(glob.glob("/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/*.zip"))

['/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/archive.zip']


In [52]:
# extract the archive zip file to folder called imdb_files
with ZipFile("/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/archive.zip", "r") as file:
    file.extractall(path="/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files")

In [53]:
# check if the files have been extracted
csv_files = glob.glob("/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/*.csv")
for csv_file in csv_files:
    print(csv_file)

/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/adventure_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/drama_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/fantasy_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/history_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/biography_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/crime_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/sci-fi_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/music_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/western_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/action_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw_data/imdb/imdb_files/thriller_series.csv
/home/anees/projects/EDA_and_data_cleaning/raw

In [54]:
# read the each csv file into a pandas dataframe, then combine them to form a single dataframe
dataframes = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dataframes.append(df)
imdb_df = pd.concat(dataframes, ignore_index=True)

In [55]:
# get the basic information about the dataset
imdb_df.info()

# preview the dataset
imdb_df.head()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236828 entries, 0 to 236827
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Title            236828 non-null  object 
 1   IMDb ID          236828 non-null  object 
 2   Release Year     236819 non-null  object 
 3   Genre            236828 non-null  object 
 4   Cast             235956 non-null  object 
 5   Synopsis         236828 non-null  object 
 6   Rating           236828 non-null  float64
 7   Runtime          216983 non-null  object 
 8   Certificate      169091 non-null  object 
 9   Number of Votes  236828 non-null  object 
 10  Gross Revenue    45611 non-null   object 
dtypes: float64(1), object(10)
memory usage: 19.9+ MB


,Title,IMDb ID,Release Year,Genre,Cast,Synopsis,Rating,Runtime,Certificate,Number of Votes,Gross Revenue
0,The Little Mermaid,tt5971474,I) (2023,"Adventure, Family, Fantasy","Director:, Rob Marshall, | , Stars:, Halle...",A young mermaid makes a deal with a sea witch ...,7.2,135 min,PG,69638,NaN
1,Spider-Man: Across the Spider-Verse,tt9362722,2023,"Animation, Action, Adventure","Directors:, Joaquim Dos Santos, , Kemp Powers,...","Miles Morales catapults across the Multiverse,...",9.1,140 min,PG,71960,NaN
2,FUBAR,tt13064902,2023–,"Action, Adventure, Thriller","Stars:, Arnold Schwarzenegger, , Monica Barbar...",A C.I.A. operative on the edge of retirement d...,6.5,NaN,TV-MA,15422,NaN
3,Fast X,tt5433140,2023,"Action, Adventure, Crime","Director:, Louis Leterrier, | , Stars:, Vi...",Dom Toretto and his family are targeted by the...,6.3,141 min,PG-13,39326,NaN
4,Guardians of the Galaxy Vol. 3,tt6791350,2023,"Action, Adventure, Comedy","Director:, James Gunn, | , Stars:, Chris P...","Still reeling from the loss of Gamora, Peter Q...",8.2,150 min,PG-13,160447,NaN


next is to perform columnar operations on the dataset

In [ ]:
# give a name to the index column 
imdb_df.index.name = 'index'
print(type(imdb_df))

# preview the data again
imdb_df.head()

In [ ]:
# index starts at 0, set to 1
imdb_df.index = imdb_df.index + 1
print(type(imdb_df))

# preview the dataframe again
imdb_df.head()

In [ ]:
# replace space between columns names with underschore (_) and change the column names to lower case
imdb_df.columns = imdb_df.columns.str.replace(' ', '_').str.lower()
imdb_df.head()

In [ ]:
# check the data types of columns 
imdb_df.dtypes

In [ ]:
# change columns to numeric datatypes 
imdb_df[["release_year", "number_of_votes", "gross_revenue"]] = \
imdb_df[["release_year", "number_of_votes", "gross_revenue"]].apply(pd.to_numeric)

# verify the changes has been effected
imdb.dtypes

got a value error: "ValueError: Unable to parse string "I) (2023" at position 0"
lets remove all special characters from the release_year column 

In [ ]:
imdb_df["release_year"] = imdb_df["release_year"].apply(lambda x: x[1:] if x.startswith('I') else x)

In [ ]:
# change columns to numeric datatypes 
imdb_df[["release_year", "number_of_votes", "gross_revenue"]] = \
imdb_df[["release_year", "number_of_votes", "gross_revenue"]].apply(pd.to_numeric)

# verify the changes has been effected
imdb.dtypes

In [ ]:
# check for duplicate in the title column as each title must be unique
imdb_df["synopsis"].duplicated().sum()


In [ ]:
imdb_df["title"].duplicated().sum()